### Importing necessary libraries

In [1]:
from pyulog import ULog
import pandas as pd
import os
import cv2
import numpy as np

from utilities.PX4CSVPlotter import PX4CSVPlotter
from tqdm import tqdm


### Environment setup and data paths

In [2]:
DATA_DIR = "../data/1/"
video_path = os.path.join(DATA_DIR, "mp4.mp4")
output_path = os.path.join(DATA_DIR, "output.mp4")
ulog_path = os.path.join(DATA_DIR, "ulg.ulg")
csv_path = os.path.join(DATA_DIR, "csv")
SAVE_EVERY_N = 1
PLOT_EVERY_N = 10
cap = cv2.VideoCapture(video_path)
FPS = cap.get(cv2.CAP_PROP_FPS)
cap.release()


### Unpacking ULog data and saving to CSV files

In [3]:
ulog = ULog(ulog_path)
for d in ulog.data_list:
    df = pd.DataFrame(d.data)
    print(d.name)


MONITOR_CSV = [
    "vehicle_attitude",
    "sensor_accel_0.csv",
    "sensor_gyro_0.csv",
    "sensor_mag_0.csv",
    "sensor_baro_0.csv",
    "sensor_gps_0.csv",
    "vehicle_local_position_0.csv",
    "vehicle_global_position_0.csv"
]


for data in ulog.data_list:
    if data.name in MONITOR_CSV:
        df = pd.DataFrame(data.data)

        # Convert PX4 timestamp to seconds
        if "timestamp" in df.columns:
            df["timestamp_s"] = df["timestamp"] * 1e-6

        filename = f"{data.name}_{data.multi_id}.csv"
        filepath = os.path.join(DATA_DIR, "csv", filename)
        df.to_csv(filepath, index=False)
        print(f"Saved {filename}")


actuator_armed
actuator_motors
actuator_outputs
battery_status
config_overrides
control_allocator_status
cpuload
distance_sensor_mode_change_request
ekf2_timestamps
esc_status
estimator_aid_src_baro_hgt
estimator_aid_src_fake_hgt
estimator_aid_src_fake_pos
estimator_aid_src_gnss_hgt
estimator_aid_src_gnss_pos
estimator_aid_src_gnss_vel
estimator_aid_src_gravity
estimator_aid_src_mag
estimator_baro_bias
estimator_event_flags
estimator_gps_status
estimator_innovation_test_ratios
estimator_innovation_variances
estimator_innovations
estimator_sensor_bias
estimator_states
estimator_status
estimator_status_flags
event
failsafe_flags
failure_detector_status
gripper
home_position
hover_thrust_estimate
landing_gear
manual_control_setpoint
mission_result
navigator_mission_item
navigator_status
parameter_update
position_setpoint_triplet
rate_ctrl_status
rtl_status
rtl_time_estimate
sensor_accel
sensor_baro
sensor_combined
sensor_gps
sensor_gyro
sensor_gyro_fft
sensor_mag
sensor_selection
sensors_

### Importing and processing telemetry data to np.arrays

In [4]:
plotter = PX4CSVPlotter(csv_path)

all_data = plotter.plot_all(plot=False)
angles = all_data["att"]
gps = all_data["gps"]

gps_time, gps_lon, gps_lat, gps_alt = gps
gps_alt = np.array(gps_alt)

yaw_att  = np.array(angles[0])
pitch_att = np.array(angles[1])
roll_att   = np.array(angles[2])
time_att  = np.array(angles[3])

roll_norm  = (roll_att + 180) % 360 - 180
pitch_norm = (pitch_att + 180) % 360 - 180
yaw_norm   = (yaw_att + 180) % 360 - 180

### Normalisation of telemetry data length to gps_time length

In [5]:
M = len(yaw_norm)
N = M

idx = np.linspace(0, M-1, N).astype(int)

gps_time = np.arange(N)
yaw_norm   = yaw_norm[idx]
pitch_norm = pitch_norm[idx]
roll_norm  = roll_norm[idx]

old_len = len(gps_alt)

x_old = np.linspace(0, 1, old_len)
x_new = np.linspace(0, 1, N)

gps_alt = np.interp(x_new, x_old, gps_alt)


print(gps_time.shape)
print(yaw_norm.shape)
print(pitch_norm.shape)
print(roll_norm.shape)

(64409,)
(64409,)
(64409,)
(64409,)


### Saving video frames to np.array

In [6]:
cap = cv2.VideoCapture(video_path)
frames = []
frame_idx = 0

# Get total frames for tqdm progress
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

with tqdm(total=total_frames, desc="Reading frames") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % SAVE_EVERY_N == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)

        frame_idx += 1
        pbar.update(1)

cap.release()

frames = np.array(frames)
print("Frames shape:", frames.shape)


Reading frames: 100%|██████████| 5969/5969 [00:50<00:00, 118.06it/s]


Frames shape: (5969, 960, 1280, 3)


### Helper function to detect start and end of motion in video

In [8]:


# --- Parameters ---
THRESH_VIDEO = 1        # motion threshold (tune if needed)
MIN_STATIC_FRAMES = 2  # how many frames must stay static to confirm end

prev_frame = None
motion = []

frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is None:
        prev_frame = gray
        frame_idx += 1
        continue

    # compute difference
    diff = cv2.absdiff(gray, prev_frame)
    mean_diff = diff.mean()
    motion.append(mean_diff)

    prev_frame = gray
    frame_idx += 1

cap.release()

motion = np.array(motion)

# --- Detect start ---
start_frame = np.argmax(motion > THRESH_VIDEO)

# --- Detect end ---
# Find last index where motion exceeds threshold
last_motion = np.where(motion > THRESH_VIDEO)[0]
end_frame = last_motion[-1] + MIN_STATIC_FRAMES if len(last_motion) else 0


start_frame -=1

end_time = end_frame / FPS

start_time = start_frame / FPS

print("Start frame index:", start_frame)
print("End frame index:", end_frame)
print("Start time (s):", start_time)
print("End time (s):", end_time)
print("Duration (s):", end_time - start_time)

Start frame index: -1
End frame index: 5527
Start time (s): -0.07722700619869326
End time (s): 426.8336632601776
Duration (s): 426.9108902663763


### cutting video frames and telemetry data to match motion period, and normalizing lengths to be the same, preparing for overlay

In [9]:
VIDEO_START_MESS = 22.31860479142235       # video start time in seconds
VIDEO_END_MESS = 457.5700117272575        # video end time in seconds
TELEMETRY_START_IDX = 16970
TELEMETRY_END_IDX = 63418

frames_cut = frames[int(VIDEO_START_MESS*FPS):int(VIDEO_END_MESS*FPS)]

gps_time_cut = gps_time[TELEMETRY_START_IDX:TELEMETRY_END_IDX]
yaw_cut      = yaw_norm[TELEMETRY_START_IDX:TELEMETRY_END_IDX]
pitch_cut    = pitch_norm[TELEMETRY_START_IDX:TELEMETRY_END_IDX]
roll_cut     = roll_norm[TELEMETRY_START_IDX:TELEMETRY_END_IDX]
gps_alt_cut  = gps_alt[TELEMETRY_START_IDX:TELEMETRY_END_IDX]

M = len(frames_cut)
N = M
idx = np.linspace(0, len(yaw_cut) - 1, N).astype(int)

gps_time_cut = np.arange(N)
yaw_cut      = yaw_cut[idx]
pitch_cut    = pitch_cut[idx]
roll_cut     = roll_cut[idx]

old_len = len(gps_alt_cut)
x_old = np.linspace(0, 1, old_len)
x_new = np.linspace(0, 1, N)

gps_alt_cut = np.interp(x_new, x_old, gps_alt_cut)

print("gps_time_cut:", gps_time_cut.shape)
print("yaw_cut:", yaw_cut.shape)
print("pitch_cut:", pitch_cut.shape)
print("roll_cut:", roll_cut.shape)
print("gps_alt_cut:", gps_alt_cut.shape)
print("frames_cut:", frames_cut.shape)

gps_time_cut: (5636,)
yaw_cut: (5636,)
pitch_cut: (5636,)
roll_cut: (5636,)
gps_alt_cut: (5636,)
frames_cut: (5636, 960, 1280, 3)


### Debug visualization of telemetry data overlaid on video frames

In [10]:
N = min(len(frames_cut), len(gps_time_cut), len(gps_alt_cut), len(pitch_cut), len(roll_cut))

T = 1 / 30

for i in range(N):
    frame = frames_cut[i].copy()
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    text = (
        f"Idx: {i} | "
        f"Time: {gps_time_cut[i]:.2f}s | "
        f"Alt: {gps_alt_cut[i]:.2f}m | "
        f"Pitch: {pitch_cut[i]:.2f} | "
        f"Roll: {roll_cut[i]:.2f}"
    )

    cv2.putText(
        frame_bgr,
        text,
        (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (0, 255, 0),
        2
    )

    cv2.imshow("Telemetry Video", frame_bgr)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    cv2.waitKey(int(T * 1000))

cv2.destroyAllWindows()
